# Hexacene cost estimate

In [1]:
import numpy as np
import pyscf
from pyscf import gto
from IPython.display import clear_output

/home/mitarai/anaconda/envs/qulacs/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


Get $\pi$-orbitals using PiOS and calculate Hamiltonian

In [2]:
from pyscf.mcscf.PiOS import MakePiOS
basis = 'sto3g'
mol = gto.M(atom="hexacene.xyz",
            basis=basis)
mf = mol.RHF.run()
e_rhf = mf.e_tot
nuc_energy = mf.energy_nuc()
pyscf.mp.MP2(mf).run()
from pi_orbital_hamiltonian import get_PiOS_hamiltonian_integrals
active_start, active_end, one_body_integrals, two_body_integrals = \
    get_PiOS_hamiltonian_integrals(mol)
clear_output() # PiOS prints a lot of information so we clear it here

In [3]:
c_indices = []
for i, a in enumerate(mol._atom):
    if a[0] == "C":
        c_indices.append(i+1)
ncore, nactive, nvirtual, nElec, COrbNew = MakePiOS(mol, mf, c_indices)
clear_output() # PiOS prints a lot of information so we clear it here

Make an active space Hamiltonian by openfermion

In [4]:
from openfermion.ops.representations import get_active_space_integrals
from local_hamiltonian import integrals_to_InteractionOp
from openfermion.transforms import jordan_wigner
# compute active space hamiltonian
e_core, one_body_integrals_new, two_body_integrals_new =\
        get_active_space_integrals(
        one_body_integrals, np.asarray(two_body_integrals.transpose(0, 2, 3, 1), order='C'),
        occupied_indices=list(range(0, active_start)),
        active_indices=list(range(active_start, active_end))
    )
active_space_hamiltonian = integrals_to_InteractionOp(e_core, one_body_integrals_new, two_body_integrals_new)
active_space_hamiltonian = jordan_wigner(active_space_hamiltonian)

Make the pauli coefficient vector

In [5]:
def coef2array(h):
    _array = np.array([h.terms[key] for key in h.terms])
    return _array[np.where(np.abs(_array)>1e-11)]
coef_vec_active_only = coef2array(active_space_hamiltonian)

The total Hamiltonian $H_{\mathrm{total}}$ is partitioned into $H$ and $V$ in such a way that, if $\sigma_\ell$ contained in $H_{\mathrm{total}}$ has any Pauli-$X$ or $Y$ operators acting on inactive orbitals, $\sigma_\ell$ is grouped into $V$, and otherwise, $\sigma_\ell$ is taken into $H$. Here, we extract Pauli coefficients of $H$.

In [6]:
from utils import get_active_space_total_hamiltonian_coefs_from_integrals
from utils import get_inactive_space_total_hamiltonian_coefs_from_integrals
from utils import get_hamiltonian_coefs_from_integrals
result = get_active_space_total_hamiltonian_coefs_from_integrals(
    one_body_integrals, two_body_integrals, active_start, active_end
    )
coef_vec = np.array([])
for i, coef in enumerate(result):
    if i == 0 or i == len(result)-1:
        coef_vec = np.append(coef_vec, coef)
    else:
        coef_vec = np.append(coef_vec, (coef, coef))

result = get_inactive_space_total_hamiltonian_coefs_from_integrals(
    one_body_integrals, two_body_integrals, active_start, active_end
    )
coef_vec_inactive = np.array([])
for i, coef in enumerate(result):
    if i == 0 or i == len(result)-1:
        coef_vec_inactive = np.append(coef_vec_inactive, coef)
    else:
        coef_vec_inactive = np.append(coef_vec_inactive, (coef, coef))

In [7]:
all_coef_vec = get_hamiltonian_coefs_from_integrals(one_body_integrals, two_body_integrals)

Get norms of vectors

In [8]:
h1norm = np.sum(np.abs(coef_vec[1:])) + abs(coef_vec[0]+nuc_energy-e_rhf)
h1norm_active_space_only = np.sum(np.abs(coef_vec_active_only)[1:])
v1norm = np.sum(np.abs(coef_vec_inactive[1:]))
v23norm = np.sum(np.abs(coef_vec_inactive[1:])**(2/3))**(3/2)
print(h1norm)
print(h1norm_active_space_only)
print(v1norm)
print(v23norm)
lvlh = len(coef_vec_inactive[np.where(np.abs(coef_vec_inactive) > 1e-8)])/len(coef_vec[np.where(np.abs(coef_vec) > 1e-8)])
print(lvlh)

3404.8835568534437
311.1663299218101
22685.026994163727
149167641.59335476
385.67549910467017


# Generate values in the paper

In [9]:
from cost_estimate import get_cost_for_first_order_pert, get_cost_for_second_order_pert
Delta = 0.033
# MP2 of pentacene using RHF as reference yields energy correction of about 1 Hartree. We therefore have to set delta0 smaller than chem acc.
delta0 = 0.3e-3*Delta/2. 
delta1 = 0.3e-3
delta2 = 0.3e-3
r_scale = 1
epsilon_scale = 1
n_subsystems = 1
p = 0.7 # this is just an expectation.
print(get_cost_for_first_order_pert(h1norm, v23norm, v1norm, delta0, delta1, Delta, p, r_scale=r_scale, epsilon_scale=epsilon_scale, n_subsystems=n_subsystems))
print(get_cost_for_second_order_pert(h1norm, v23norm, v1norm, delta0, delta1, Delta, p, r_scale=r_scale, epsilon_scale=epsilon_scale, n_subsystems=n_subsystems))

epsilon:  1.0100441353089018e-09
n_filter:  (23710446.086245816+0j)
kappa:  9.6905076044632e-06
M1:  2267901542.6787386
r:  6.612291007570374e-10
x_th: 1.4537942098009499e-09
(2.8060795828992497e+18+0j)
r:  9.618926321135126e-16
epsilon_filter:  1.469315265693494e-15
epsilon_ptb:  1.984928731237441e-10
kappa:  9.6905076044632e-06
w:  9.6905076044632e-06
w0:  1.4537942098009499e-09
M2:  6.835318721794499e+22
n_filter:  (38654744.20789895+0j)
n_ptb:  (175552437.75850233+0j)
(2.347700095251335e+32+0j)
